In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Preprocess Data to Label Churn listings

In [4]:
import os

base = 'data/historic_airbnb/nyc/'
dataframes = []

for filename in os.listdir(base):
    dataframes.append(pd.read_csv(base + filename))

/Users/jonathandinu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/jonathandinu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/jonathandinu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (40,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/jonathandinu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interact

In [5]:
joined = pd.concat(dataframes)
len(joined)

/Users/jonathandinu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


464565

In [6]:
sum([len(df) for df in dataframes ])

464565

In [7]:
joined.id.value_counts()

1332570     15
3847681     15
2120677     15
2165878     15
3767267     15
            ..
307163       1
11457547     1
690206       1
4692875      1
7346179      1
Name: id, Length: 77201, dtype: int64

In [8]:
joined[joined.id == 4632705].last_scraped

20438    2016-05-02
21707    2016-02-02
16845    2016-01-02
8282     2015-06-01
19002    2015-09-02
20247    2016-04-03
19469    2015-01-02
20522    2015-11-20
18826    2015-08-01
18891    2015-10-01
8301     2015-05-02
20936    2015-12-02
17930    2015-04-02
31153    2015-11-02
17694    2015-03-02
Name: last_scraped, dtype: object

In [9]:
joined.last_scraped = pd.to_datetime(joined.last_scraped)

In [10]:
# get most recent
dedup = joined.sort_index(by='last_scraped').drop_duplicates(subset=['id'], keep='last')

/Users/jonathandinu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  


In [11]:
dedup.last_scraped

19444   2015-01-01
1520    2015-01-01
1517    2015-01-01
21691   2015-01-01
21706   2015-01-01
           ...    
21723   2016-05-02
21724   2016-05-02
21725   2016-05-02
21712   2016-05-02
0       2016-05-02
Name: last_scraped, Length: 77201, dtype: datetime64[ns]

In [12]:
len(dedup)

77201

In [13]:
# sanity checks
len(joined.id.unique())

77201

### Churn:

* Listing removed
* No review in last year

In [14]:
# Label Listings that have been removed (not present in most recent scrape)
dedup['churn'] = dedup.last_scraped != pd.Timestamp('2016-05-02')
dedup['churn'].value_counts()

True     44626
False    32575
Name: churn, dtype: int64

In [15]:
dedup.columns

Index(['accommodates', 'amenities', 'availability_30', 'availability_365',
       'availability_60', 'availability_90', 'bathrooms', 'bed_type',
       'bedrooms', 'beds', 'calculated_host_listings_count',
       'calendar_last_scraped', 'calendar_updated', 'cancellation_policy',
       'city', 'cleaning_fee', 'country', 'country_code', 'description',
       'experiences_offered', 'extra_people', 'first_review',
       'guests_included', 'has_availability', 'host_about',
       'host_acceptance_rate', 'host_has_profile_pic', 'host_id',
       'host_identity_verified', 'host_is_superhost', 'host_listing_count',
       'host_listings_count', 'host_location', 'host_name',
       'host_neighbourhood', 'host_picture_url', 'host_response_rate',
       'host_response_time', 'host_since', 'host_thumbnail_url',
       'host_total_listings_count', 'host_url', 'host_verifications', 'id',
       'instant_bookable', 'is_location_exact', 'jurisdiction_names',
       'last_review', 'last_scraped', 'l

## Feature Transformation

In [16]:
features = dedup[['host_is_superhost', 'price',
                    'room_type', 'accommodates',
                    'review_scores_cleanliness', 'review_scores_location', 'review_scores_rating', 'host_identity_verified',
                    'host_response_time', 'host_response_rate', 'host_has_profile_pic', 'availability_365',
                    'calculated_host_listings_count', 'calendar_updated', 'last_review', 'minimum_nights',
                    'number_of_reviews', 'reviews_per_month', 'neighbourhood', 'churn']]

features['last_review'] = pd.to_datetime(features.last_review)
features['days_since_last_review'] = (features.last_review.apply(lambda x: pd.Timestamp('2016-05-02') - x)/ np.timedelta64(1, 'D'))
features['host_response_rate'] = features.host_response_rate.str.strip('%').astype(float)
features['price'] = features.price.str.strip('$').str.replace(',', '').astype(float)

/Users/jonathandinu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jonathandinu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/jonathandinu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [17]:
features = features.drop('last_review', axis=1)

features = features.fillna({'host_is_superhost': 'f', 'reviews_per_month': 0, 'instant_bookable': 'f',
                          'review_scores_cleanliness': 0, 'review_scores_rating': 0, 'review_scores_location': 0, 'host_identity_verified': 'f',
                         'host_identity_verified': 'f', 'host_response_rate': 0, 'host_response_time': 'a few days or more', 
                         'host_has_profile_pic': 'f', 'calculated_host_listings_count': 1, 'days_since_last_review': 3650,
                         'availability_365': features.availability_365.mean()})

In [18]:
features.head()

,host_is_superhost,price,room_type,accommodates,review_scores_cleanliness,review_scores_location,review_scores_rating,host_identity_verified,host_response_time,host_response_rate,host_has_profile_pic,availability_365,calculated_host_listings_count,calendar_updated,minimum_nights,number_of_reviews,reviews_per_month,neighbourhood,churn,days_since_last_review
19444,f,200.0,Entire home/apt,3,10.0,10.0,99.0,f,a few days or more,0.0,f,243.0,1.0,4 months ago,7,4,0.0,Williamsburg,True,850.0
1520,f,450.0,Entire home/apt,4,10.0,10.0,94.0,f,a few days or more,0.0,f,208.0,1.0,2 weeks ago,3,7,0.0,Tribeca,True,505.0
1517,f,28.0,Shared room,1,0.0,0.0,0.0,f,a few days or more,0.0,f,364.0,1.0,6 weeks ago,1,0,0.0,Soundview,True,3650.0
21691,f,48.0,Private room,2,8.0,6.0,80.0,f,a few days or more,0.0,f,143.0,1.0,4 months ago,1,1,0.0,Harlem,True,612.0
21706,f,160.0,Private room,2,0.0,0.0,0.0,f,a few days or more,0.0,f,365.0,1.0,2 months ago,1,0,0.0,Tribeca,True,3650.0


In [19]:
features.to_csv('data/churn.csv')